In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from requests import get
import tensorflow as tf
import xgboost as xgb
import os

In [ ]:
pets = ['PET_AMMONITE', 'PET_ARMADILLO', 'PET_BABY_YETI', 'PET_BAL','PET_BAT','PET_BEE','PET_BLACK_CAT','PET_BLAZE','PET_BLUE_WHALE','PET_CHICKEN','PET_DOLPHIN','PET_ELEPHANT','PET_ENDER_DRAGON','PET_ENDERMAN','PET_ENDERMITE','PET_FLYING_FISH','PET_GHOUL','PET_GIRAFFE','PET_GOLDEN_DRAGON','PET_GOLEM','PET_GRIFFIN','PET_GUARDIAN','PET_HORSE','PET_HOUND','PET_JELLYFISH','PET_JERRY','PET_LION','PET_MAGMA_CUBE','PET_MEGALODON','PET_MITHRIL_GOLEM','PET_MONKEY','PET_MOOSHROOM_COW','PET_OCELOT','PET_PARROT','PET_PHOENIX','PET_PIG','PET_PIGMAN','PET_RABBIT','PET_RAT','PET_REINDEER','PET_ROCK','PET_SCATHA','PET_SHEEP','PET_SILVERFISH','PET_SKELETON','PET_SKELETON_HORSE','PET_SNAIL','PET_SNOWMAN','PET_SPIDER','PET_SPIRIT','PET_SQUID','PET_TARANTULA','PET_TIGER','PET_TURTLE','PET_FROST_WISP','PET_WITHER_SKELETON','PET_WOLF','PET_ZOMBIE']

In [ ]:
petsData = []

for i in pets:
    for j in range(1):
        res = get(
            f'https://sky.coflnet.com/api/auctions/tag/{i}/sold?page={j}&pageSize=1000&token=64wcAGNBYE8tMW7w9W2vWq')
        petsData.extend(res.json())

In [ ]:
petsData = list(filter(lambda elm: not isinstance(elm, str), petsData))

In [ ]:
print(len(petsData))

In [ ]:
raw_df = pd.DataFrame(petsData)
# df.columns

In [ ]:
def organize_data(df):
    price, start_time, end_time = df['highestBidAmount'], df['start'],  df['end']
    # start_time = pd.to_datetime(start_time, format='mixed').astype('int64')
    end_time = pd.to_datetime(end_time, format='mixed').astype('int64')

    nbtData = pd.json_normalize(df['flatNbt'])

    nbtData = nbtData.drop(columns=['uuid', 'uid', 'bossId', 'hideRightClick', 'spawnedFor', 'active', 'blaze_kills'])

    nbtData['candyUsed'] = nbtData['candyUsed'].fillna(0)
    nbtData = nbtData.fillna('')

    nbtData.type = nbtData.type.astype('category')
    nbtData.skin = nbtData.skin.astype('category')
    nbtData.heldItem = nbtData.heldItem.astype('category')
    nbtData.tier = nbtData.tier.astype('category')

    nbtData.exp = nbtData.exp.astype(np.float32)
    nbtData.hideInfo = nbtData.hideInfo.astype(bool)
    nbtData.candyUsed = nbtData.candyUsed.astype(int)

    return pd.concat([nbtData, price, end_time], axis=1)

In [ ]:
df = organize_data(raw_df)

In [ ]:
def get_median(df, item):
    all_items = df[(df['type'] == item.values[0][0]) & (df['tier'] == item.values[0][2])]
    return all_items['highestBidAmount'].median()


In [ ]:
bst = xgb.Booster()

In [ ]:
bst.load_model('./model.json')